# Load the Documents

In [1]:
import sys
sys.path.append("../")
from src.config import Configuration
from src.prepare import data_load

doc_loader = data_load.DocDataLoader()
config = Configuration()

folder = "test_training_program"
folder = "test_major"
db = doc_loader.load_docs(data_folder=folder,size=460, overlap=20)
db = doc_loader.load_major_docs_full(data_folder=folder)

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


From (44, 4) to Counter({2: 37, 1: 6, 3: 1})
From (44, 4) to 44


# Choose model

### Gemini

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

INDEX = "gemini-major"
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",
                                          google_api_key=config.load_gemini_token())

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### VN SBert

In [3]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
INDEX = "vnsbert-major"

MODEL = "keepitreal/vietnamese-sbert"
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=conf.load_hg_token(0),
    model_name=MODEL)

### LaBSE

In [4]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
INDEX = "labse-uni"

MODEL = "sentence-transformers/LaBSE"

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=config.load_hg_token(0),
    model_name=MODEL)

# Push vector to  ES

In [4]:
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore

es = ElasticsearchStore.from_documents(db, embedding=embeddings,
                          es_connection=config.load_elasticsearch_connection(),
                          index_name=INDEX,
                          distance_strategy="EUCLIDEAN_DISTANCE")


# Push text data to ES

In [5]:
INDEX = "text-split-training_program"
INDEX = "text-full-major"

k1 = 1.5
b = 0.75

# Define the index settings and mappings
settings = {
    "analysis": {"analyzer": {"default": {"type": "standard"}}},
    "similarity": {
        "custom_bm25": {
            "type": "BM25",
            "k1": k1,
            "b": b,
        }
    },
}
mappings = {
    "properties": {
        "content": {
            "type": "text",
            "similarity": "custom_bm25",  # Use the custom BM25 similarity
        }
    }
}

# Create the index with the specified settings and mappings
es_connect = config.load_elasticsearch_connection()
es_connect.indices.create(index=INDEX, mappings=mappings, settings=settings)
len(db)

44

In [6]:
from langchain.retrievers import ElasticSearchBM25Retriever
from src.custom.es_bm25_retriever import MyElasticSearchBM25Retriever
retriever = MyElasticSearchBM25Retriever(client=es_connect, index_name=INDEX)

In [7]:
len(db)

44

In [8]:
a = retriever.add_documents(documents=db)

## Load

In [9]:
retriever.k = 2
a = retriever.get_relevant_documents("Hóa học")
a

[Document(page_content='Việt Nam học - Chuyên ngành Việt ngữ học và văn hóa xã hội Việt Nam là một ngành khoa học nghiên cứu về đất nước, con người Việt Nam theo hướng tiếp cận liên ngành. Chương trình đào tạo ngành Việt Nam học, chuyên ngành Việt ngữ học và Văn hóa xã hội Việt Nam tại Trường Đại học Tôn Đức Thắng là chương trình dành cho đối tượng là người nước ngoài, nhằm đáp ứng nhu cầu về nguồn nhân lực là người nước ngoài sử dụng thông thạo tiếng Việt, am hiểu về đất nước và con người Việt Nam.\n\nSau khi tốt nghiệp ngành Việt Nam học, chuyên ngành Việt ngữ học và Văn hóa xã hội Việt Nam, học viên có thể tìm kiếm việc làm phù hợp ở các cơ quan ngoại giao, tổ chức quốc tế, công ty du lịch, văn phòng thương mại, tổ chức chính phủ và phi chính phủ có nhu cầu sử dụng nguồn nhân lực am hiểu về Việt Nam (ở Việt Nam và nước ngoài). Ngoài ra sinh viên cũng có thể làm công tác phiên dịch, biên dịch, nghiên cứu và giảng dạy tiếng Việt tại các trường đại học, cao đẳng hoặc các trung tâm ngôn

In [12]:
retriever.k = 3
a = retriever.get_relevant_documents("em muốn biết ngành kỹ thuật phần mềm")
a


[Document(page_content='Thiết kế thời trang là ngành đào tạo nhân sự có khả năng thiết kế, sản xuất quần áo và phụ kiện phục vụ các nhu cầu của xã hội. Ngoài các kỹ năng thông thường như sáng tạo, phác hoạ và sử dụng các phần mềm đồ họa chuyên nghiệp để làm cho mẫu phục trang trở nên sống động, ngành học còn đào tạo các nhà thiết kế thời trang trở thành những người tiên phong, đi đầu và dự đoán trước những xu hướng mới về ngành thiết kế lẫn xu hướng thời trang trong những năm tiếp theo.\n\nSinh viên theo học ngành Thiết kế thời trang tại TDTU có đầy đủ năng lực để tự tin ứng tuyển vào các vị trí như: Nhà thiết kế (Fashion Designer), giám đốc sáng tạo (Creative Director) tại các doanh nghiệp trong nước và quốc tế; biên tập viên thời trang (Fashion Editor), người tạo phong cách (Stylist) trong các tạp chí thời trang, tạp chí mốt; tạo lập công ty hoặc thương hiệu riêng hoặc trở thành cán bộ giảng dạy, cán bộ nghiên cứu tại các cơ sở giáo dục đại học, cao đẳng và viện nghiên cứu liên quan.